In [1]:
#
# https://github.com/rawlings-group/paresto/blob/master/examples/green_book/bvsm.m
#
# I had trouble fitting relative error sum(lc/lc_pred - 1)^2 as done in the example
# So I chose minimizing sum(lc - lc_pred)^2. Still I had trouble with collocation. But finite difference
# discretization worked.

In [2]:
# Import libraries
from pyomo.environ import *
from pyomo.dae import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.stats as spstat
from pyomo.contrib.interior_point.inverse_reduced_hessian import inv_reduced_hessian_barrier

In [3]:
# load flow rate data
flow_data = pd.read_csv('flow_data.csv')
lc_data = pd.read_csv('lc_data.csv')
teaf      = 0.00721
teaden    = 0.728
cBf = teaf

In [4]:
tQf = np.insert(flow_data['t'].values, 0, 0)
Qf = np.insert(flow_data["Qf"].values / teaden, 0, 0)

In [5]:
tlc = lc_data['t'].values
lc = lc_data['lc_meas'].values

In [6]:
tout = np.sort(np.unique(np.concatenate((tQf, tlc))))

In [32]:
max(tout), len(tout)

(869, 118)

In [8]:
Qf_if = interp1d(tQf, Qf, 'previous', bounds_error = False)
lc_if = interp1d(tlc, lc, 'previous', bounds_error = False)

In [9]:
data = [{'tout': tout, 'tlc': tlc, 'Qf_if': Qf_if, 'lc':lc, 'cBf': cBf}]

In [61]:
def bvsm_model(data):
    
    tout = data['tout']
    tlc = data['tlc']
    Qf_if = data['Qf_if']
    lc_m = data['lc']
    cBf = data['cBf']
    
    Vr0 = 2370
    nB0 = 0
    nC0 = 0
    nD0 = 0
    
    m = ConcreteModel()
    
    m.k1 = Var(initialize = 2000, bounds = (500, 5000))
    m.k2 = Var(initialize = 1000, bounds = (500, 5000))
    m.nA0 = Var(initialize = 2.35, bounds = (2, 3))
    
    m.time = ContinuousSet(bounds = (0, max(tout)), initialize = tout)
    
    m.Vr = Var(m.time)
    m.nA = Var(m.time, initialize = 2.35, bounds = (0, 2.35))
    m.nB = Var(m.time)
    m.nC = Var(m.time)
    m.nD = Var(m.time)
    m.y = Var(m.time, initialize = 0.5, bounds = (0, 1))
    
    m.dVr = DerivativeVar(m.Vr)
    m.dnA = DerivativeVar(m.nA)
    m.dnB = DerivativeVar(m.nB)
    m.dnC = DerivativeVar(m.nC)
    m.dnD = DerivativeVar(m.nD)
    
    def _dVr_eq(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dVr[t] == float(Qf_if(t))
    m.dVr_eq = Constraint(m.time, rule = _dVr_eq)
    
    def _dnA_eq(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dnA[t] == -m.k1 * m.nA[t] * m.nB[t] / m.Vr[t]
    m.dnA_eq = Constraint(m.time, rule = _dnA_eq)

    def _dnB_eq(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dnB[t] == float(Qf_if(t)) * cBf - (m.k1 * m.nA[t] * m.nB[t] + m.k2 * m.nB[t] * m.nC[t]) / m.Vr[t]
    m.dnB_eq = Constraint(m.time, rule = _dnB_eq)
    
    def _dnC_eq(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dnC[t] == (m.k1 * m.nA[t] * m.nB[t] - m.k2 * m.nB[t] * m.nC[t]) / m.Vr[t]
    m.dnC_eq = Constraint(m.time, rule = _dnC_eq)
    
    def _dnD_eq(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dnD[t] == m.k2 * m.nB[t] * m.nC[t] / m.Vr[t]
    m.dnD_eq = Constraint(m.time, rule = _dnD_eq)
    
    def _ycalc(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.y[t] == m.nC[t] / (m.nC[t] + 2 * m.nD[t] + 1.0e-6)
    m.ycalc = Constraint(m.time, rule = _ycalc)
    
    def init_nA_rule(m):
        return m.nA[m.time.first()] == m.nA0
    m.init_nA = Constraint(rule = init_nA_rule)

    def SSE_rule(m):
        return sum((float(lc_if(t)) / m.y[t] - 1.0) ** 2 for t in tlc) 
#        return sum((float(lc_if(t)) - m.y[t]) ** 2 for t in tlc) 
    m.SSE_Objective = Objective(rule=SSE_rule, sense=minimize)
    
    #disc = TransformationFactory('dae.collocation')
    #disc.apply_to(m, nfe=200, ncp=2)
    
    disc = TransformationFactory('dae.finite_difference')
    disc.apply_to(m, nfe=150, scheme = 'BACKWARD')
    
    for t in m.time:
        if t == 0:
            m.Vr[t].fix(Vr0)
            m.nB[t].fix(nB0)
            m.nC[t].fix(nC0)
            m.nD[t].fix(nD0)
        else:
            m.Vr[t] = Vr0
            m.Vr[t].setlb(Vr0)
            m.nB[t] = nB0
            m.nB[t].setlb(nB0)
            m.nB[t].setub(2.35)
            m.nC[t] = nC0
            m.nC[t].setlb(nC0)
            m.nC[t].setub(2.35)
            m.nD[t] = nD0
            m.nD[t].setlb(nD0)
            m.nD[t].setub(2.35)
        
    return m

In [62]:
m = bvsm_model(data[0])

In [ ]:
m.Vr.pprint()
#data[0]['tout']

In [63]:
solver = SolverFactory('ipopt')
solver.solve(m, tee = True)

Ipopt 3.14.5: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     6448
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2285

Total number of variables............................:     1654
                     variables with only lower bounds:      150
                variables with lower and upper bounds:      754
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1651
Total number of inequ

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1651, 'Number of variables': 1654, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.5\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.21822023391723633}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [64]:
[m.k1(), m.k2(), m.nA0()]

[2200.1819830807212, 926.7832342537837, 2.3292893352016577]

In [65]:
solve_result, inv_red_hes = inv_reduced_hessian_barrier(m, 
                    independent_variables= [m.k1, m.k2, m.nA0],
                    tee=True)

Ipopt 3.14.5: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     6448
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2285

Total number of variables............................:     1654
                     variables with only lower bounds:      150
                variables with lower and upper bounds:      754
                     variables with only upper bounds:        0
Total number of equality constraints...

In [66]:
[m.k1(), m.k2(), m.nA0()]

[2200.149240435383, 926.7702139390947, 2.329289289148926]

In [67]:
inv_red_hes

array([[ 7.82685510e+06,  3.17845887e+05, -5.98176475e+01],
       [ 3.17845887e+05,  2.05271105e+04, -3.64865672e+00],
       [-5.98176475e+01, -3.64865672e+00,  6.52389703e-04]])

In [68]:
n = len(data[0]['tlc'])
p = 3.0
sse = m.SSE_Objective()
n, p, sse

(35, 3.0, 0.05536241498514171)

In [69]:
mult_factor = p * spstat.f.ppf(0.95, p, n-p)
mult_factor

8.703358751522511

In [70]:
cov_est = 2 * sse / (n - p) * inv_red_hes
cov_est

array([[ 2.70821000e+04,  1.09979474e+03, -2.06978089e-01],
       [ 1.09979474e+03,  7.10269006e+01, -1.26249030e-02],
       [-2.06978089e-01, -1.26249030e-02,  2.25736684e-06]])

In [71]:
delta_param = np.sqrt(mult_factor * np.diag(cov_est))
delta_param

array([4.85494832e+02, 2.48630770e+01, 4.43245682e-03])